<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.utils import *
from delta import *
import hashlib
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("NB_DIM_Wallet") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Spark will automatically use the master specified in spark-defaults.conf
spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark

In [3]:
trgt_path_processed = "/mnt/Wallet/Wallet_Parquet"
trgt_path_csv = "/mnt/Wallet/Wallet_Processed"
source_path = "/data/Project_Files/Template.xlsm"
sheet_name ='SPENDING_HISTORY'


In [4]:
# Read the Excel file (use Spark-Excel library)
df = pd.read_excel(source_path, sheet_name = sheet_name)
df = spark.createDataFrame(df)
df= df.withColumn("Date", to_date(df["Date"],"yyyy-MM-dd"))

In [5]:
df.createOrReplaceTempView("vw_src")
query = "select distinct `Wallet used` from vw_src"
print(query)
df_src = spark.sql(query)
df_src.show()

select distinct `Wallet used` from vw_src


+--------------------+
|         Wallet used|
+--------------------+
|               G-pay|
|       Amazon Wallet|
|           ICICI Pay|
|Credit Card - Amazon|
+--------------------+



In [6]:
df_output = \
df_src.withColumn("Walletsk",xxhash64("Wallet used")).withColumnRenamed("Wallet used","WalletUsed")

In [7]:
df_output.createOrReplaceTempView("vw_source")

In [8]:
column_name = df_output.columns
set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
query = f"MERGE INTO delta.`{trgt_path_processed}` AS target USING vw_source AS source ON target.Walletsk = source.Walletsk WHEN MATCHED THEN UPDATE SET {set_clause}"
print(query)

MERGE INTO delta.`/mnt/Wallet/Wallet_Parquet` AS target USING vw_source AS source ON target.Walletsk = source.Walletsk WHEN MATCHED THEN UPDATE SET target.WalletUsed = source.WalletUsed, target.Walletsk = source.Walletsk


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    query = f"MERGE INTO delta.`{trgt_path_processed}` AS target USING vw_source AS source ON target.Walletsk = source.Walletsk WHEN MATCHED THEN UPDATE SET {set_clause}"
    spark.sql(query)
else :
    query=f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source"
    spark.sql(query)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51060)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/soc

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o33.sql

In [ ]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)